# 🧪 Lab Supply Calculator

Welcome! Use this tool to:

- Select a protocol
- Enter your sample size
- Click "Run All"
- Receive a downloadable CSV with supply needs and reorder status

🔒 **Note**: You'll need a GitHub access token to access the files. Please see Aubrie or Vicki for this.

---

## 🧭 Instructions

1. Enter your **GitHub token**
2. Enter the **sample size**
3. Choose the **protocol**
4. Click **Runtime → Run All**
5. Download the CSV report when prompted

---

### Example:
- Token: `ghp_XXXXXXXXXXXXXXXXXXXX`
- Sample size: `50`


In [ ]:
#@title Install packages and imports


##Install packages and imports
# Installing packages
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import yaml
except ImportError:
    install_package("pyyaml")
    import yaml

# Import all required libraries
import requests
import csv
import pandas as pd
import os
from io import StringIO
from google.colab import files
from IPython.display import display, clear_output
import ipywidgets as widgets
import time

print("✅ All packages installed and imported successfully!")

# Define GitHub repo configuration
GITHUB_USER = "aono87"
REPO_NAME = "inventory_test"
BRANCH = "main"
INVENTORY_PATH = "test_protocol_check/inventory/inventory.csv"
PROTOCOL_DIR = "test_protocol_check/protocols"

print("✅ Configuration set")

#Helper Functions
def github_raw_url(path):
    return f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/{BRANCH}/{path}"

def fetch_file(path, token):
    url = github_raw_url(path)
    headers = {"Authorization": f"token {token}"}
    try:
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        return r.text
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching {path}: {e}")
        raise

def get_protocol_list(token, user, repo, path, branch="main"):
    url = f"https://api.github.com/repos/{user}/{repo}/contents/{path}?ref={branch}"
    headers = {"Authorization": f"token {token}"}
    try:
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        files = r.json()
        return [f['path'] for f in files if f['name'].endswith('.yaml')]
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching protocol list: {e}")
        return []

def load_inventory(text):
    inv = {}
    try:
        # Remove BOM if present
        text = text.lstrip('\ufeff')
        reader = csv.DictReader(StringIO(text))
        for row in reader:
            item = row['Item'].strip()
            inv[item] = {
                'unit': row['Unit'].strip(),
                'stock': float(row['Stock Quantity']),
                'threshold': float(row['Reorder Threshold'])
            }
        return inv
    except Exception as e:
        print(f"❌ Error loading inventory: {e}")
        return {}

def load_protocol(text):
    try:
        return yaml.safe_load(text)
    except Exception as e:
        print(f"❌ Error loading protocol: {e}")
        return {}

def calculate_needs(protocol_data, sample_count):
    if 'supplies_per_sample' not in protocol_data:
        print("❌ Protocol missing 'supplies_per_sample' section")
        return {}
    return {item: qty * sample_count for item, qty in protocol_data['supplies_per_sample'].items()}

print("✅ Helper functions defined")

✅ All packages installed and imported successfully!
✅ Configuration set
✅ Helper functions defined


In [64]:
#@title Input github token and sample size
print("📝 Please enter your details below:")

# Create input widgets
token_input = widgets.Text(
    value='',
    placeholder='Enter your GitHub token here (ghp_...)',
    description='GitHub token:',
    layout=widgets.Layout(width='80%'),
    style={'description_width': 'initial'}
)

sample_input = widgets.IntText(
    value=50,
    description='Sample size:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Create a button to proceed
proceed_button = widgets.Button(
    description='Continue with these settings',
    disabled=False,
    button_style='success',
    tooltip='Click to proceed with the entered token and sample size',
    layout=widgets.Layout(width='300px', margin='10px 0px')
)

# Status output
status_output = widgets.Output()

# Function to handle button click
def on_button_click(b):
    with status_output:
        status_output.clear_output()
        if not token_input.value or len(token_input.value) < 10:
            print("❌ Please enter a valid GitHub token")
            return
        if sample_input.value <= 0:
            print("❌ Please enter a valid sample size")
            return
        print(f"✅ Token received: {token_input.value[:4]}...")
        print(f"✅ Sample size: {sample_input.value}")
        print("✅ Ready to proceed to next step!")
        proceed_button.disabled = True
        proceed_button.description = "Settings confirmed ✓"

proceed_button.on_click(on_button_click)

# Display all widgets
display(token_input, sample_input, proceed_button, status_output)

print("👆 Enter your GitHub token and sample size above, then click 'Continue' to proceed")
print("🔄 You can now generate a report by pressing play button on the left side of the section below called 'Complete Inventory Check'")


📝 Please enter your details below:


Text(value='', description='GitHub token:', layout=Layout(width='80%'), placeholder='Enter your GitHub token h…

IntText(value=50, description='Sample size:', style=DescriptionStyle(description_width='initial'))

Button(button_style='success', description='Continue with these settings', layout=Layout(margin='10px 0px', wi…

Output()

👆 Enter your GitHub token and sample size above, then click 'Continue' to proceed
🔄 You can now generate a report by pressing play button on the left side of the section below called 'Complete Inventory Check'


In [65]:
#@title Complete Processing Pipeline - Run All Safe!

def process_inventory_check():
    """Main processing function that handles everything"""

    # Check if inputs are provided
    if not token_input.value or len(token_input.value) < 10:
        print("⏳ Waiting for GitHub token...")
        print("   Please enter your GitHub token in the cell above")
        return False

    if sample_input.value <= 0:
        print("⏳ Waiting for valid sample size...")
        print("   Please enter a sample size > 0 in the cell above")
        return False

    print("✅ Inputs received!")
    print(f"   Token: {token_input.value[:4]}...")
    print(f"   Sample size: {sample_input.value}")

    try:
        # Step 1: Fetch available protocols
        print("\n🔍 Step 1: Fetching available protocols...")
        protocols = get_protocol_list(
            token_input.value,
            GITHUB_USER,
            REPO_NAME,
            PROTOCOL_DIR,
            BRANCH
        )

        if not protocols:
            print("❌ No protocols found, using defaults")
            protocols = [
                "test_protocol_check/protocols/dna_extraction_mn.yaml",
                "test_protocol_check/protocols/pcr_setup.yaml"
            ]
        else:
            print(f"✅ Found {len(protocols)} protocols")

        # Step 2: Create protocol dropdown
        print("\n🔧 Step 2: Creating protocol selector...")
        protocol_dropdown = widgets.Dropdown(
            options=protocols,
            description='Protocol:',
            layout=widgets.Layout(width='95%'),
            style={'description_width': 'initial'}
        )

        display(protocol_dropdown)
        print("✅ Protocol dropdown created!")
        print("👆 Please select your protocol above")

        # Step 3: Create processing button
        process_button = widgets.Button(
            description='Generate Report',
            disabled=False,
            button_style='primary',
            tooltip='Click to generate the supply report',
            layout=widgets.Layout(width='200px', margin='10px 0px')
        )

        # Output area for results
        output_area = widgets.Output()

        def generate_report(b):
            """Generate the supply report"""
            with output_area:
                output_area.clear_output()

                try:
                    print("🔄 Generating report...")

                    # Fetch data files
                    print("📥 Fetching inventory data...")
                    inventory_text = fetch_file(INVENTORY_PATH, token_input.value)

                    print("📥 Fetching protocol data...")
                    protocol_text = fetch_file(protocol_dropdown.value, token_input.value)

                    # Parse data
                    print("🔍 Parsing data...")
                    inventory = load_inventory(inventory_text)
                    protocol = load_protocol(protocol_text)

                    if not inventory:
                        raise Exception("Failed to load inventory data")
                    if not protocol:
                        raise Exception("Failed to load protocol data")

                    # Calculate needs
                    needs = calculate_needs(protocol, sample_input.value)

                    if not needs:
                        raise Exception("Failed to calculate supply needs")

                    print(f"✅ Calculated needs for {len(needs)} items")

                    # Create report
                    report_rows = []
                    per_sample = protocol.get("supplies_per_sample", {})

                    for item, total_required in needs.items():
                        per_unit = per_sample.get(item, "N/A")
                        inv = inventory.get(item)

                        if not inv:
                            report_rows.append({
                                "Item": item,
                                "Per Sample": per_unit,
                                "Need": total_required,
                                "Stock": "N/A",
                                "Status": "MISSING",
                                "Reorder": "⚠️"
                            })
                        else:
                            stock = inv['stock']
                            threshold = inv['threshold']
                            status = "OK" if stock >= total_required else "LOW"
                            reorder_flag = "YES" if (stock - total_required) < threshold else "NO"
                            report_rows.append({
                                "Item": item,
                                "Per Sample": per_unit,
                                "Need": round(total_required, 2),
                                "Stock": round(stock, 2),
                                "Status": status,
                                "Reorder": reorder_flag
                            })

                    # Convert to DataFrame
                    df = pd.DataFrame(report_rows)

                    # Display report
                    protocol_name = os.path.basename(protocol_dropdown.value)
                    print(f"\n📋 Supply Report for {sample_input.value} samples using {protocol_name}")
                    print("=" * 80)
                    display(df)

                    # Generate and download CSV
                    print("\n💾 Generating CSV download...")

                    # Create metadata
                    metadata_info = [
                        ["Protocol:", protocol_name],
                        ["Sample Size:", sample_input.value],
                        ["Generated:", pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")],
                        [""], # Blank row
                    ]

                    # Save to CSV
                    output_path = "/content/supply_report.csv"

                    with open(output_path, 'w', newline='', encoding='utf-8') as f:
                        writer = csv.writer(f)
                        for row in metadata_info:
                            writer.writerow(row)

                    # Append DataFrame
                    df.to_csv(output_path, mode='a', index=False)

                    print("✅ Report generated successfully!")
                    print("📁 Starting download...")

                    # Download file
                    files.download(output_path)
                    print("🎉 Download complete!")

                    # Update button
                    process_button.description = "Report Generated ✓"
                    process_button.button_style = "success"
                    process_button.disabled = True

                except Exception as e:
                    print(f"❌ Error: {e}")
                    print("Please check your inputs and try again.")

        # Connect button to function
        process_button.on_click(generate_report)

        # Display button and output area
        display(process_button, output_area)

        print("\n✅ Everything is ready!")
        print("👆 Select your protocol and click 'Generate Report' to continue")

        return True

    except Exception as e:
        print(f"❌ Setup error: {e}")
        return False

# Run the main processing function
print("🚀 Starting Lab Supply Calculator...")
print("=" * 50)

# This will run when the cell executes but wait for inputs
success = process_inventory_check()

if not success:
    print("\n⏸️  Process paused - waiting for inputs")
    print("   Fill in your details above and re-run this cell")
else:
    print("\n🎯 Ready to generate your supply report!")

🚀 Starting Lab Supply Calculator...
✅ Inputs received!
   Token: ghp_...
   Sample size: 15

🔍 Step 1: Fetching available protocols...
✅ Found 1 protocols

🔧 Step 2: Creating protocol selector...


Dropdown(description='Protocol:', layout=Layout(width='95%'), options=('test_protocol_check/protocols/dna_extr…

✅ Protocol dropdown created!
👆 Please select your protocol above


Button(button_style='primary', description='Generate Report', layout=Layout(margin='10px 0px', width='200px'),…

Output()


✅ Everything is ready!
👆 Select your protocol and click 'Generate Report' to continue

🎯 Ready to generate your supply report!
